In [15]:
!pip install bs4
!pip install crypto
!pip install pycryptodome
!pip install pycryptodomex

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import datetime
from Cryptodome.Cipher.AES import new, MODE_GCM, MODE_CBC
from Crypto.Protocol.KDF import PBKDF2
from urllib.parse import unquote, unquote_to_bytes
from bs4 import BeautifulSoup
import sqlite3
import secretstorage
"""
bus = secretstorage.dbus_init()
collection = secretstorage.get_default_collection(bus)
for item in collection.get_all_items():
    if item.get_label() == 'Chrome Safe Storage':
        my_pass = item.get_secret()
        break
"""

# Default values used by both Chrome browsers under Linux
salt = b'saltysalt'
iv = b' ' * 16
length = 16
my_pass = 'peanuts'.encode('utf8')
iterations = 1


def get_time(ticks):
  t = datetime.datetime.now() + datetime.timedelta(microseconds = ticks/10)
  return t.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

# Function to get rid of padding
def clean(x):
    return x[:-x[-1]].decode('utf8')

def linux_decrypt_value(value, encrypted_value):
    if value or (encrypted_value[:3] not in (b'v10',b'v11')):
      return value
    keyit=encrypted_value[:3]
    if keyit==b'v10':  # peanuts flavor
      # Encrypted cookies prefixed with 'v10' according to the
      # Chromium code. Strip it off.
      my_pass='peanuts'.encode('utf8')
    else:  # keyring flavor
      # the key is on the keyring
      my_pass=b't1KGF0OfYUO/x5r0gg0hXw=='
    encrypted_value = encrypted_value[3:]
    key = PBKDF2(my_pass, salt, length, iterations)
    cipher = new(key, MODE_CBC, IV=iv)
    decrypted = cipher.decrypt(encrypted_value)
    return clean(decrypted)

In [18]:

bools = ['FALSE', 'TRUE']
sql_ = """SELECT
  host_key,
  is_httponly httponly,
  path,
  is_secure secure,
  expires_utc/10000000 exp_utc,
  name,
  value val_,
  encrypted_value eval_
FROM
  cookies;
  """

filename='/content/drive/My Drive/Cookies'
lstat='/content/drive/My Drive/lstat'

with sqlite3.connect(filename) as conn:
  #conn.enable_load_extension(True)
  curs =  conn.cursor()
  curs.execute(sql_)
  results = curs.fetchall()

  key = PBKDF2(my_pass, salt, length, iterations)

  for r in results:
    # only the MIT entries
    if 'emeritus' in r[0].lower():
      host_key, httponly, path, secure, exp_utc, cookie_name, val, enc_val = r
      if enc_val is None or len(enc_val) == 0:
        continue
      httponly = bools[httponly]
      secure = bools[secure]
      """
      data = enc_val
      nonce = data[3:15]
      ciphertext = data[15:-16]
      tag = data[-16:]
      decrypted = ''
      """
      try:
        #cipher = new(key, MODE_GCM,  nonce=nonce)
        #plaintext = cipher.decrypt_and_verify(ciphertext, tag)
        #decrypted = plaintext.decode('utf8')
        decrypted=linux_decrypt_value(val, enc_val)
      except Exception as e:
        print('ERR:',e)
        continue
      tt = get_time(exp_utc)
      print(f'''--header "{cookie_name}: {decrypted}; " \\''')

  curs.close()


--header "user_uuid: 5babbcdd-33a1-4cb1-a006-fec80cc23615; " \
--header "rce.htmleditor: RAW; " \
--header "_zendesk_shared_session: -R0g4eWxQZzhMajhZTjZ0RDJxRlhGUWZsR25Tb0JRaUw4NzRodDVRa1FDamtDRUF6dEFlajNkcWNyd1BybjcwN0FHc3BwNzVWM0VTN2c4dG1KbDBZcldUZVNqWWc3M3BIVjhib0I1eU1wQytUSURNUkQrWWUwY3BBYTBjMTZTcWw5azIwNWt1Mlh0a1JYSWdmWVFaWVRnPT0tLXYzU2FzdndKS0I3bW91QmpGT0Zrb1E9PQ%3D%3D--e8598e40e49af78f620295745160193413bd239e; " \
--header "preferences|{}: {"global":{"playbackRate":1,"volume":1,"cc_lang":"en"},"videos":{"d9e5f124-0ea4-44eb-a2f0-949420604743":{"lastKnownTime":499.053817},"f1296a32-0f8c-4f0d-be74-042123c8b6f5":{"lastKnownTime":772.88531},"d46affc0-9359-484c-8571-433878803d74":{"lastKnownTime":197.017842},"ca7397c0-8ef6-4c79-9357-38b4fcd15c92":{"lastKnownTime":196.707582},"bb91c845-7e96-4af4-96c1-9644d75edc72":{"lastKnownTime":76.701518},"7fc0135f-4e2d-48f2-bc49-fa4a2e0a1131":{"lastKnownTime":108.974752},"bce62182-634c-4465-a1de-63639c0f9e08":{"lastKnownTime":10.65858},"1cf26270-b